In [1]:
import pandas as pd
import numpy as np
import calc

In [2]:
# OHLCVデータをロード
df = pd.read_csv('bybit_ethusdt_20230101-20230330.csv', index_col=0)

In [3]:
for i in range(10):
    period = i + 1
    col = f'log_return_{period}'
    df[col] = calc.add_log_returns(df, 'Close', period)

In [4]:
# 短期および長期の単純移動平均 (SMA) を計算する関数
def calculate_sma(dataframe, column, window):
    return dataframe[column].rolling(window=window).mean()

# 短期および長期のSMAを計算して新たなカラムとして追加
short_term_window = 5  # 短期SMAのウィンドウ期間（例：5分）
long_term_window = 20  # 長期SMAのウィンドウ期間（例：20分）

df['SMA_short_term'] = calculate_sma(df, 'Close', short_term_window)
df['SMA_long_term'] = calculate_sma(df, 'Close', long_term_window)

# 結果を表示
print(df.tail())

                  Timestamp     Open     High      Low    Close  Volume  \
128867  2023-03-31 02:47:00  1803.98  1804.82  1803.98  1804.82  529.98   
128868  2023-03-31 02:48:00  1804.82  1805.45  1804.81  1805.36  394.25   
128869  2023-03-31 02:49:00  1805.36  1805.36  1804.50  1804.51  414.99   
128870  2023-03-31 02:50:00  1804.51  1804.51  1802.67  1802.67  271.97   
128871  2023-03-31 02:51:00  1802.67  1802.67  1801.95  1802.65  100.22   

        log_return_1  log_return_2  log_return_3  log_return_4  log_return_5  \
128867      0.000466      0.001192      0.001558      0.001286      0.000665   
128868      0.000299      0.000765      0.001491      0.001857      0.001585   
128869     -0.000471     -0.000172      0.000294      0.001020      0.001386   
128870     -0.001020     -0.001491     -0.001192     -0.000726      0.000000   
128871     -0.000011     -0.001031     -0.001502     -0.001203     -0.000738   

        log_return_6  log_return_7  log_return_8  log_return_9  log_

In [5]:
# ボリンジャーバンドの上限、下限、中心線を計算する関数
def calculate_bollinger_bands(dataframe, column, window, num_std_dev):
    sma = dataframe[column].rolling(window=window).mean()
    std_dev = dataframe[column].rolling(window=window).std()
    
    upper_band = sma + (std_dev * num_std_dev)
    lower_band = sma - (std_dev * num_std_dev)
    
    return upper_band, lower_band, sma

# ボリンジャーバンドの上限、下限、中心線を計算して新たなカラムとして追加
window = 20  # ボリンジャーバンドのウィンドウ期間（例：20分）
num_std_dev = 2  # 標準偏差の倍数（通常は2）

df['BB_upper'], df['BB_lower'], df['BB_middle'] = calculate_bollinger_bands(df, 'Close', window, num_std_dev)

# 結果を表示
print(df.tail())

                  Timestamp     Open     High      Low    Close  Volume  \
128867  2023-03-31 02:47:00  1803.98  1804.82  1803.98  1804.82  529.98   
128868  2023-03-31 02:48:00  1804.82  1805.45  1804.81  1805.36  394.25   
128869  2023-03-31 02:49:00  1805.36  1805.36  1804.50  1804.51  414.99   
128870  2023-03-31 02:50:00  1804.51  1804.51  1802.67  1802.67  271.97   
128871  2023-03-31 02:51:00  1802.67  1802.67  1801.95  1802.65  100.22   

        log_return_1  log_return_2  log_return_3  log_return_4  ...  \
128867      0.000466      0.001192      0.001558      0.001286  ...   
128868      0.000299      0.000765      0.001491      0.001857  ...   
128869     -0.000471     -0.000172      0.000294      0.001020  ...   
128870     -0.001020     -0.001491     -0.001192     -0.000726  ...   
128871     -0.000011     -0.001031     -0.001502     -0.001203  ...   

        log_return_6  log_return_7  log_return_8  log_return_9  log_return_10  \
128867      0.000770      0.000854      0

In [6]:
# RSIを計算する関数
def calculate_rsi(dataframe, column, window):
    delta = dataframe[column].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# RSIを計算して新たなカラムとして追加
window = 14  # RSIのウィンドウ期間（例：14分）

df['RSI'] = calculate_rsi(df, 'Close', window)

# 結果を表示
print(df.tail())

                  Timestamp     Open     High      Low    Close  Volume  \
128867  2023-03-31 02:47:00  1803.98  1804.82  1803.98  1804.82  529.98   
128868  2023-03-31 02:48:00  1804.82  1805.45  1804.81  1805.36  394.25   
128869  2023-03-31 02:49:00  1805.36  1805.36  1804.50  1804.51  414.99   
128870  2023-03-31 02:50:00  1804.51  1804.51  1802.67  1802.67  271.97   
128871  2023-03-31 02:51:00  1802.67  1802.67  1801.95  1802.65  100.22   

        log_return_1  log_return_2  log_return_3  log_return_4  ...  \
128867      0.000466      0.001192      0.001558      0.001286  ...   
128868      0.000299      0.000765      0.001491      0.001857  ...   
128869     -0.000471     -0.000172      0.000294      0.001020  ...   
128870     -0.001020     -0.001491     -0.001192     -0.000726  ...   
128871     -0.000011     -0.001031     -0.001502     -0.001203  ...   

        log_return_7  log_return_8  log_return_9  log_return_10  \
128867      0.000854      0.000604      0.000843       

In [7]:
# MACDおよびシグナルラインを計算する関数
def calculate_macd(dataframe, column, short_window, long_window, signal_window):
    short_ema = dataframe[column].ewm(span=short_window).mean()
    long_ema = dataframe[column].ewm(span=long_window).mean()

    macd = short_ema - long_ema
    signal_line = macd.ewm(span=signal_window).mean()

    return macd, signal_line

# MACDおよびシグナルラインを計算して新たなカラムとして追加
short_window = 12  # 短期EMAのウィンドウ期間（例：12分）
long_window = 26   # 長期EMAのウィンドウ期間（例：26分）
signal_window = 9  # シグナルラインのウィンドウ期間（例：9分）

df['MACD'], df['Signal_line'] = calculate_macd(df, 'Close', short_window, long_window, signal_window)

# 結果を表示
print(df.tail())

                  Timestamp     Open     High      Low    Close  Volume  \
128867  2023-03-31 02:47:00  1803.98  1804.82  1803.98  1804.82  529.98   
128868  2023-03-31 02:48:00  1804.82  1805.45  1804.81  1805.36  394.25   
128869  2023-03-31 02:49:00  1805.36  1805.36  1804.50  1804.51  414.99   
128870  2023-03-31 02:50:00  1804.51  1804.51  1802.67  1802.67  271.97   
128871  2023-03-31 02:51:00  1802.67  1802.67  1801.95  1802.65  100.22   

        log_return_1  log_return_2  log_return_3  log_return_4  ...  \
128867      0.000466      0.001192      0.001558      0.001286  ...   
128868      0.000299      0.000765      0.001491      0.001857  ...   
128869     -0.000471     -0.000172      0.000294      0.001020  ...   
128870     -0.001020     -0.001491     -0.001192     -0.000726  ...   
128871     -0.000011     -0.001031     -0.001502     -0.001203  ...   

        log_return_9  log_return_10  SMA_short_term  SMA_long_term  \
128867      0.000843       0.001075        1803.196 

In [8]:
# ヒストリカルボラティリティを計算する関数
def calculate_historical_volatility(dataframe, column, window):
    log_returns = np.log(dataframe[column] / dataframe[column].shift(1))
    historical_volatility = log_returns.rolling(window=window).std() * np.sqrt(252) * 100

    return historical_volatility

# ヒストリカルボラティリティを計算して新たなカラムとして追加
window = 14  # ヒストリカルボラティリティのウィンドウ期間（例：14分）

df['Historical_volatility'] = calculate_historical_volatility(df, 'Close', window)

# 結果を表示
print(df.tail())

                  Timestamp     Open     High      Low    Close  Volume  \
128867  2023-03-31 02:47:00  1803.98  1804.82  1803.98  1804.82  529.98   
128868  2023-03-31 02:48:00  1804.82  1805.45  1804.81  1805.36  394.25   
128869  2023-03-31 02:49:00  1805.36  1805.36  1804.50  1804.51  414.99   
128870  2023-03-31 02:50:00  1804.51  1804.51  1802.67  1802.67  271.97   
128871  2023-03-31 02:51:00  1802.67  1802.67  1801.95  1802.65  100.22   

        log_return_1  log_return_2  log_return_3  log_return_4  ...  \
128867      0.000466      0.001192      0.001558      0.001286  ...   
128868      0.000299      0.000765      0.001491      0.001857  ...   
128869     -0.000471     -0.000172      0.000294      0.001020  ...   
128870     -0.001020     -0.001491     -0.001192     -0.000726  ...   
128871     -0.000011     -0.001031     -0.001502     -0.001203  ...   

        log_return_10  SMA_short_term  SMA_long_term     BB_upper  \
128867       0.001075        1803.196      1803.1780 

In [9]:
# フィボナッチリトレースメントを計算する関数
def calculate_fibonacci_retracements(high, low):
    retracement_levels = [0.0, 0.236, 0.382, 0.5, 0.618, 0.786, 1.0]
    retracements = {}
    
    for level in retracement_levels:
        retracements[level] = high - (high - low) * level

    return retracements

# データフレーム内の特定の範囲に対するリトレースメント水準を計算
range_start = 100  # 範囲の開始インデックス
range_end = 200    # 範囲の終了インデックス

high = df.loc[range_start:range_end, 'High'].max()
low = df.loc[range_start:range_end, 'Low'].min()

retracements = calculate_fibonacci_retracements(high, low)

# リトレースメント水準を表示
print("Fibonacci retracement levels:")
for level, value in retracements.items():
    print(f"{level * 100}%: {value}")

Fibonacci retracement levels:
0.0%: 1204.2
23.599999999999998%: 1202.8902
38.2%: 1202.0799
50.0%: 1201.4250000000002
61.8%: 1200.7701000000002
78.60000000000001%: 1199.8377
100.0%: 1198.65


In [10]:
# ストキャスティクスを計算する関数
def calculate_stochastics(dataframe, high_col, low_col, close_col, k_window, d_window):
    highest_high = dataframe[high_col].rolling(window=k_window).max()
    lowest_low = dataframe[low_col].rolling(window=k_window).min()
    
    stoch_k = (dataframe[close_col] - lowest_low) / (highest_high - lowest_low) * 100
    stoch_d = stoch_k.rolling(window=d_window).mean()

    return stoch_k, stoch_d

# ストキャスティクスを計算して新たなカラムとして追加
k_window = 14  # %Kのウィンドウ期間（例：14分）
d_window = 3   # %Dのウィンドウ期間（例：3分）

df['%K'], df['%D'] = calculate_stochastics(df, 'High', 'Low', 'Close', k_window, d_window)

# 結果を表示
print(df.tail())

                  Timestamp     Open     High      Low    Close  Volume  \
128867  2023-03-31 02:47:00  1803.98  1804.82  1803.98  1804.82  529.98   
128868  2023-03-31 02:48:00  1804.82  1805.45  1804.81  1805.36  394.25   
128869  2023-03-31 02:49:00  1805.36  1805.36  1804.50  1804.51  414.99   
128870  2023-03-31 02:50:00  1804.51  1804.51  1802.67  1802.67  271.97   
128871  2023-03-31 02:51:00  1802.67  1802.67  1801.95  1802.65  100.22   

        log_return_1  log_return_2  log_return_3  log_return_4  ...  \
128867      0.000466      0.001192      0.001558      0.001286  ...   
128868      0.000299      0.000765      0.001491      0.001857  ...   
128869     -0.000471     -0.000172      0.000294      0.001020  ...   
128870     -0.001020     -0.001491     -0.001192     -0.000726  ...   
128871     -0.000011     -0.001031     -0.001502     -0.001203  ...   

        SMA_long_term     BB_upper     BB_lower  BB_middle        RSI  \
128867      1803.1780  1804.701425  1801.654575  

In [11]:
# オンバランスボリュームを計算する関数
def calculate_obv(dataframe, close_col, volume_col):
    obv = [0]
    
    for i in range(1, len(dataframe)):
        if dataframe[close_col][i] > dataframe[close_col][i-1]:
            obv.append(obv[-1] + dataframe[volume_col][i])
        elif dataframe[close_col][i] < dataframe[close_col][i-1]:
            obv.append(obv[-1] - dataframe[volume_col][i])
        else:
            obv.append(obv[-1])
            
    return obv

# オンバランスボリュームを計算して新たなカラムとして追加
df['OBV'] = calculate_obv(df, 'Close', 'Volume')

# 結果を表示
print(df.tail())

                  Timestamp     Open     High      Low    Close  Volume  \
128867  2023-03-31 02:47:00  1803.98  1804.82  1803.98  1804.82  529.98   
128868  2023-03-31 02:48:00  1804.82  1805.45  1804.81  1805.36  394.25   
128869  2023-03-31 02:49:00  1805.36  1805.36  1804.50  1804.51  414.99   
128870  2023-03-31 02:50:00  1804.51  1804.51  1802.67  1802.67  271.97   
128871  2023-03-31 02:51:00  1802.67  1802.67  1801.95  1802.65  100.22   

        log_return_1  log_return_2  log_return_3  log_return_4  ...  \
128867      0.000466      0.001192      0.001558      0.001286  ...   
128868      0.000299      0.000765      0.001491      0.001857  ...   
128869     -0.000471     -0.000172      0.000294      0.001020  ...   
128870     -0.001020     -0.001491     -0.001192     -0.000726  ...   
128871     -0.000011     -0.001031     -0.001502     -0.001203  ...   

           BB_upper     BB_lower  BB_middle        RSI      MACD  Signal_line  \
128867  1804.701425  1801.654575  1803.17

In [12]:
df.to_csv('bybit_ethusdt_20230101-20230330_features.csv')